# Drone Routing

This script solves a drone routing problem using Pyomo for the `wt_1000` weight distribution.

## Summary

- Uses `Centroids.csv` and `Drone_Links_Filtered_1000.csv`, which are created using `Network_Setup.ipynb`.
- Assigns customer nodes to drones, minimizing energy while respecting payload and battery limits
- Depot is fixed at node `9999`

## Notes

- This handles **one of six total weight scenarios**.
- To run other scenarios, change `wt_1000` to:
  - `wt_2000`, `wt_3000`, `wt_4000`, `wt_5000`, `wt_6000`
- Also update the links file (e.g., `Drone_Links_Filtered_2000.csv`, etc.)

## Efficiency

To reduce solve time, we did the following:
- **Clustering**: Average distance to 20 nearest neighbors approximates intra-tour costs
- **Depot averaging**: System-wide depot penalty approximates depot travel without full routing

## Output

Prints drone assignments, payload (kg), and energy (J). Repeat for other weight scenarios by adjusting the inputs as noted.


In [1]:
# First attempt: wt_1000

from pyomo.environ import *
import pandas as pd

# Loading data
centroids = pd.read_csv("Centroids.csv")
links = pd.read_csv("Drone_Links_Filtered_1000.csv")

# Assigning nodes and weights
depot = 9999  # assuming this is consistent in your network
nodes = centroids['cid'].tolist() + [depot]
customer_nodes = [cid for cid, wt in zip(centroids['cid'], centroids['wt_1000']) if wt > 0]

weights = {row['cid']: row['wt_1000'] for _, row in centroids.iterrows()}
weights[depot] = 0
weights_kg = {node: w * 0.453592 for node, w in weights.items()}

# Link lengths (directed)
arc_lengths = {(row['from_id'], row['to_id']): row['length'] for _, row in links.iterrows()}

# Drone & Energy Parameters
K = range(1, 151)  # allow up to 150 drones
B_max, W_max = 808*3600, 20
energy_per_meter = 80
neighbor_count = 20

# Precomputing a_i (neighbor distances)
a_vals = {}
for i in customer_nodes:
    neighbors = [j for j in centroids['cid'] if j != i and (i,j) in arc_lengths]
    neighbors = sorted(neighbors, key=lambda j: arc_lengths[(i,j)])
    chosen = neighbors[:neighbor_count]  # up to 20 nearest neighbors
    if chosen:
        a_vals[i] = sum(arc_lengths[(i,j)] for j in chosen) / len(chosen)
    else:
        a_vals[i] = 0  # isolated node, not expected but safe

# Computing system-wide depot penalty. This will be a constant. 
depot_distances = [
    arc_lengths.get((i,depot), arc_lengths.get((depot,i), 0))
    for i in centroids['cid']
    if i in customer_nodes
]
avg_b = sum(depot_distances) / len(depot_distances) if depot_distances else 0
b_penalty = 2 * avg_b  # two depot legs per drone

# Building model
model = ConcreteModel()
model.x = Var(K, customer_nodes, domain=Binary)  # node assignment
model.y = Var(K, domain=Binary)                  # drone activation

# Objective: minimize total energy (a_i sum + constant depot penalty per active drone)
model.obj = Objective(
    expr=sum(
        energy_per_meter * (
            sum(a_vals[i]*model.x[k,i] for i in customer_nodes) +
            b_penalty * model.y[k]
        )
        for k in K
    ),
    sense=minimize
)

# Constraints
# Each non-empty node must be served by exactly one drone
model.node_cover = ConstraintList()
for i in customer_nodes:
    model.node_cover.add(sum(model.x[k,i] for k in K) == 1)

# Activate drone only if it serves at least one node
model.activation = ConstraintList()
for k in K:
    model.activation.add(sum(model.x[k,i] for i in customer_nodes) <= 1000 * model.y[k])

# Payload limit per drone
model.payload = ConstraintList()
for k in K:
    model.payload.add(sum(weights_kg[i]*model.x[k,i] for i in customer_nodes) <= W_max * model.y[k])

# Energy limit (only for the variable a_i part)
model.energy_limit = ConstraintList()
for k in K:
    energy_expr = energy_per_meter * sum(a_vals[i]*model.x[k,i] for i in customer_nodes)
    model.energy_limit.add(energy_expr <= B_max * model.y[k])

# Solve
solver = SolverFactory('cplex')
solver.options['mipgap'] = 0.01      # stop once gap ≤ 1%
solver.options['timelimit'] = 600    # optional: stop after 10 minutes max (adjust as needed)
results = solver.solve(model, tee=True)

# Output
for k in K:
    if model.y[k].value > 0.5:
        assigned = [i for i in customer_nodes if model.x[k,i].value > 0.5]
        if assigned:
            a_sum = sum(a_vals[i] for i in assigned)
            total_distance = a_sum + b_penalty  # depot penalty applied once per active drone
            energy = energy_per_meter * total_distance
            payload = sum(weights_kg[i] for i in assigned)
            print(f"Drone {k}: serves {assigned}, Payload={payload:.1f} kg, Energy≈{energy:.0f} J")



Welcome to IBM(R) ILOG(R) CPLEX(R) Interactive Optimizer 22.1.1.0
  with Simplex, Mixed Integer & Barrier Optimizers
5725-A06 5725-A29 5724-Y48 5724-Y49 5724-Y54 5724-Y55 5655-Y21
Copyright IBM Corp. 1988, 2022.  All Rights Reserved.

Type 'help' for a list of available commands.
Type 'help' followed by a command name for more
information on commands.

CPLEX> Logfile 'cplex.log' closed.
Logfile 'C:\Users\msela\AppData\Local\Temp\tmpg3ltm2j1.cplex.log' open.
CPLEX> New value for mixed integer optimality gap tolerance: 0.01
CPLEX> New value for time limit in seconds: 600
CPLEX> Problem 'C:\Users\msela\AppData\Local\Temp\tmpm58t4fyt.pyomo.lp' read.
Read time = 0.08 sec. (3.67 ticks)
CPLEX> Problem name         : C:\Users\msela\AppData\Local\Temp\tmpm58t4fyt.pyomo.lp
Objective sense      : Minimize
Variables            :   45150  [Binary: 45150]
Objective nonzeros   :   45150
Linear constraints   :     750  [Less: 450,  Equal: 300]
  Nonzeros           :  180450
  RHS nonzeros       :    